In [3]:
import sys
sys.path.append('/home/aayushjoshi/ML/MLenv/lib/python3.11/site-packages')

In [4]:
import os
import cv2
import pandas as pd

dataset_path = '/home/aayushjoshi/ML/Projects/emotions/dataset/train'

data = {'image_path': [], 'label': []}

for class_label in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_label)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        data['image_path'].append(image_path)
        data['label'].append(class_label)

df = pd.DataFrame(data)

In [5]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])
test_df['encoded_label'] = label_encoder.transform(test_df['label'])

In [7]:
train_df.to_csv('train_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

label_encoder_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
pd.DataFrame(list(label_encoder_mapping.items()), columns=['Class', 'Encoded Label']).to_csv('label_mapping.csv', index=False)


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2023-12-27 14:11:02.605336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 14:11:02.605392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 14:11:02.607628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 14:11:02.618822: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-27 14:11:03.959157: W tensorflow/compiler/tf2

In [9]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label', 
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.


In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 'num_classes' is the number of emotion classes

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')

2023-12-27 14:11:39.992467: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201867264 exceeds 10% of free system memory.
2023-12-27 14:11:40.208842: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 97329152 exceeds 10% of free system memory.


  1/180 [..............................] - ETA: 2:27 - loss: 1.9370 - accuracy: 0.2188

2023-12-27 14:11:40.524937: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201867264 exceeds 10% of free system memory.
2023-12-27 14:11:40.732098: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 97329152 exceeds 10% of free system memory.


  2/180 [..............................] - ETA: 1:30 - loss: 1.9590 - accuracy: 0.1406

2023-12-27 14:11:41.032393: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201867264 exceeds 10% of free system memory.


180/180 [==============================] - 80s 441ms/step - loss: 1.9415 - accuracy: 0.1703
Test Accuracy: 0.1703239232301712


In [13]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')

180/180 [==============================] - 82s 455ms/step - loss: 1.9415 - accuracy: 0.1703
Test Accuracy: 0.1703239232301712


In [14]:
model.save('emotion_detection_model.h5')

/home/aayushjoshi/ML/MLenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
